# Recommendation Algorithm v1
Filter stock with positive EMA200 gradient, train, and predict if those stocks are reccomended or not.

Alurnya:
- buat function yang ngebuat grup dari 100 stock (tapi shufflenya selalu sama), set seed=0
- function: take group list, find time slices from those stock list. Return: time list
- function: make multiple shot input from those time slices. Return: inputs, one hot label
- function: merge multiple shot input, return: inputs, one hot label + 1 node for no recommendation

## Part 1: Fetch new data
Please use `conda env:talib`

In [1]:
import coremlv2 as core
import importlib
core._init_calc()
importlib.reload(core)

<module 'coremlv2' from 'C:\\Users\\jonat\\Documents\\#PROJECT\\idx\\coremlv2.py'>

Fetch newest data from `yfinance` and calculate v1 indicators. If this cell was run close to the market opening (9AM), the data from current trading day may fetched too, but not represent data specification. 

If that condition occured, fix dataset to match global condition.

In [2]:
# Fetch newest data
core.prepare_newest_data_v1(period='max', sleep=1.5)

# Find for incomplete date entry and fix that
core.fix_dataset_length()

0:00:00.515116 0 AHAP
- HDTX.JK: No data found, symbol may be delisted
0:00:01.754386 50 RICY
- TRIO.JK: No data found, symbol may be delisted
0:00:01.704385 100 BTEL
0:00:01.719391 150 BWPT
0:00:01.683159 200 BLTZ
0:00:01.697841 250 KBRI
0:00:01.734669 300 ROTI
- NIPS.JK: No data found, symbol may be delisted
0:00:01.743997 350 BKSL
- SUGI.JK: No data found, symbol may be delisted
- TRIL.JK: No data found, symbol may be delisted
0:00:01.681257 400 VICO
0:00:01.735108 450 SIMA
0:00:01.666641 500 PRDA
0:00:01.637289 550 TDPM
0:00:01.624484 600 ZONE
0:00:01.632371 650 AGAR
0:00:01.628368 700 HOMI


UnboundLocalError: local variable 'df' referenced before assignment

In [ ]:
# Load full id ticker list



# Load ticker list in db

# Create list of ticker in full list but absent in db

In [ ]:
origin_db = 'idx_raw.db'
target_db = 'idx_indicators.db'

calculate_all_indicator(origin_db, target_db, excluded_stock, verbose=1, selected_stock_only=selected_stock_only)

## Part 2: Input filtering, model engineering, and predict recommendation
Please use `conda env:tensorflow`

In [1]:
import coremlv2 as core
core._init_ml()
core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import importlib
importlib.reload(core)

<module 'coremlv2' from 'C:\\Users\\Jonathan Raditya\\Documents\\#PROJECT\\idx\\coremlv2.py'>

Using EMA200 gradient as stocks filter. Only select stocks that have positive EMA200_G with 0 as an entry point at first iteration. In the next iterations, sew stock added to selected list only if the gradient bigger than average gradient from the current selected list.

Dafault value: `ma=True`, new control: `ma=False`. `ma` stands for moving average, mean that the benchmark to check wheter some stock eligible to recommend or not are based on average of stock's emas in current list. If set to `False`, all stocks that have positive ema are eligible to recommend.

In [2]:
selected_tickers, selected_tickers_ema = core.get_recommended_stocks(ma=False) 
selected_tickers_array = core.np.expand_dims(core.np.array(selected_tickers), axis=1)

In [3]:
st_df = core.pd.DataFrame({'ticker':selected_tickers,'ema':selected_tickers_ema})
selected_tickers = st_df.sort_values(by='ema', ascending=False)[:10].ticker.values

In [15]:
print(list(st_df.sort_values(by='ema', ascending=False).ticker.values))

['TMAS', 'DGIK', 'PKPK', 'ADRO', 'POLY', 'MDLN', 'MTDL', 'MASA', 'GZCO', 'APIC', 'DSFI', 'BBNI', 'TLKM', 'KLBF', 'BBCA', 'PTBA', 'BMRI', 'CMNP', 'AKRA', 'SRSN', 'EXCL', 'ISAT', 'INDF', 'SMCB', 'SMSM', 'ASII', 'INDY', 'BBRI', 'BUMI', 'BUDI', 'UNSP', 'UNTR', 'TURI', 'INKP', 'CSAP']


In [2]:
# ERROR: EXCL
ticker_groups = ['ADRO', 'AKRA', 'ASII', 'BBCA', 'BBRI', 'BMRI', 'BUMI', 'CMNP', 'CSAP', 'DGIK', 'INDY', 'ISAT', 'KLBF', 'MASA', 'MTDL', 'PKPK', 'POLY', 'PTBA', 'SMSM', 'SRSN', 'TURI', 'UNSP', 'UNTR']

models_source = core.np.expand_dims(core.np.array(ticker_groups), axis=1)
models_ticker_target = core.np.expand_dims(core.np.array(ticker_groups), axis=1)
shift = 0
interval = 1
recurrent = 120
db_ver = '3'
dataset_ver = '4'
kt_iter = '1'
split = 0.8
ROOT_PATH = './'
model_base_id = '263'
modes = [['val_loss', True],['val_accuracy', False]]


models = []
standard_scaler = []

for i, model_source in enumerate(models_source):
    model_ticker_target = models_ticker_target[i]
    print(model_ticker_target)
    different_mode_models = []
    for mode in modes:
        print(f'Current mode: {mode[0]}')
        # Fetch trained model
        model, data_version = core.read_best_model(model_base_id, model_source, model_ticker_target, shift, interval, recurrent, db_ver, dataset_ver, kt_iter, split, ROOT_PATH=ROOT_PATH, retrain=False, primary_metrics=mode[0], ascending=mode[1], verbose=0)
        different_mode_models.append(model)
    models.append(different_mode_models)
    
    # Fetch standard scaler
    train_mean, train_std = core.standard_scaler_one_shot(model_ticker_target, data_version, ROOT_PATH=ROOT_PATH, split=split)
    standard_scaler.append([train_mean, train_std])

['ADRO']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_accuracy
Load model from keras_tuner
['AKRA']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_accuracy
Load model from keras_tuner
['ASII']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_accuracy
Load model from keras_tuner
['BBCA']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_accuracy
Load model from keras_tuner
['BBRI']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_accuracy
Load model from keras_tuner
['BMRI']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_accuracy
Load model from keras_tuner
['BUMI']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_accuracy
Load model from keras_tuner
['CMNP']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_accuracy
Load model from keras_tuner
['CSAP']
Current mode: val_loss
Load model from keras_tuner
Current mode: val_ac

In [3]:
standard_scaler[0][0][1]

950.971142054031

In [4]:
db_path = 'idx_indicators.db'
db_conn = core.sqlite3.connect(db_path)

prediction_results = []
for i, mode in enumerate(modes):
    # Predict for tomorrow
    # print(f'Mode: {mode[0]}')
    prediction_result = []
    for j, sticker in enumerate(core.np.squeeze(models_source)):
        tomorrow_input = core.make_one_shot_input(sticker, db_conn, recurrent, shift_from_last=0)
        tomorrow_input_n = (tomorrow_input - standard_scaler[j][0]) / standard_scaler[j][1]
        tomorrow_input_e = core.np.expand_dims(tomorrow_input_n, 0)
        current_model = models[j][i]
        tomorrow_prediction = current_model.predict(tomorrow_input_e)
        # val_acc = histories[i].history['val_accuracy'][-1]
        # print(sticker, f'val_acc:{val_acc*100:.0f}%',tomorrow_prediction)
        
        # Convert prediction to text
        tomorrow_prediction_argmax = core.np.argmax(tomorrow_prediction)
        prediction = 'UP' if tomorrow_prediction_argmax == 1 else 'DOWN'
        # print(sticker, prediction)
        prediction_result.append(prediction)
    prediction_results.append(prediction_result)

# Check double down and double up
double_up = []
double_down = []
for i, value in enumerate(prediction_results[0]):
    if prediction_results[0][i] == prediction_results[1][i]:
        if value == 'UP':
            double_up.append(ticker_groups[i])
        elif value == 'DOWN':
            double_down.append(ticker_groups[i])
print(f'Double up: {double_up}')
print(f'Double down: {double_down}')

Double up: ['ADRO', 'AKRA', 'ASII', 'BBRI', 'BMRI', 'CSAP', 'DGIK', 'ISAT', 'MASA', 'PKPK', 'PTBA']
Double down: ['BBCA', 'BUMI', 'INDY', 'POLY', 'SMSM', 'SRSN', 'TURI', 'UNTR']


In [5]:
print(tomorrow_prediction)

[[0.5216182  0.47838187]]


In [ ]:
core.merge_dataset(tickers=core.stock_metadata['ticker'].values, db_ver='4', dataset_ver='2', shuffle=True, save_in_batches=True, save_batches=16, use_itertools=True, num_of_samples=1000)

In [12]:
tomorrow_input

,open,high,low,close,Volume,change,Volume_rank,rsi14,close_EMA3,close_EMA10,...,Volume_rank_EMA3_EMA30_offset,Volume_rank_EMA3_EMA200_offset,change_EMA3,change_EMA10,change_EMA30,change_EMA200,change_EMA3_EMA10_offset,change_EMA3_EMA30_offset,change_EMA3_EMA200_offset,close_rel
5054,19056.250102,19204.355673,18908.144531,18908.144531,2294200,-0.007772,0.649725,39.433551,19076.161205,19304.685222,...,0.004341,-0.048392,-0.008724,-0.004426,-0.002966,-0.000756,0.971030,1.940740,10.545442,0.549409
5055,19006.881734,19130.303044,18611.933542,18685.986328,4676000,-0.011749,0.685440,37.334261,18881.073767,19192.194514,...,0.020204,-0.030644,-0.010236,-0.005757,-0.003533,-0.000865,0.777958,1.897313,10.834401,0.542918
5056,18685.985042,18883.459124,18587.248000,18661.300781,3025000,-0.001321,0.642857,37.097929,18771.187274,19095.668381,...,-0.001126,-0.051788,-0.005779,-0.004951,-0.003390,-0.000870,0.167235,0.704462,5.645980,0.542196
5057,18760.037822,18957.511905,18661.300781,18661.300781,1696800,0.000000,0.602195,37.097929,18716.244028,19016.692454,...,-0.038174,-0.090820,-0.002889,-0.004051,-0.003172,-0.000861,-0.286690,-0.088995,2.356387,0.542196
5058,18685.984508,18760.037287,18389.773393,18463.826172,2587700,-0.010582,0.632373,35.040066,18590.035100,18916.171312,...,-0.034285,-0.088394,-0.006736,-0.005238,-0.003650,-0.000958,0.285886,0.845538,6.034040,0.536426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5169,23550.000000,23900.000000,23425.000000,23500.000000,4935000,-0.001063,0.676152,61.990693,23211.169581,22576.903510,...,0.087303,0.033004,0.012878,0.009230,0.003648,0.000998,0.395282,2.530420,11.906295,0.683580
5170,23500.000000,23700.000000,23050.000000,23650.000000,3514900,0.006383,0.619756,63.160224,23430.584790,22772.011963,...,0.041482,-0.010310,0.009631,0.008712,0.003824,0.001051,0.105410,1.518291,8.159687,0.687963
5171,23650.000000,23700.000000,23250.000000,23250.000000,2979200,-0.016913,0.612991,58.032262,23340.292395,22858.918879,...,0.013981,-0.036793,-0.003641,0.004053,0.002486,0.000873,-1.898404,-2.464510,-5.172646,0.676275
5172,23100.000000,23475.000000,22675.000000,22850.000000,4772900,-0.017204,0.679296,53.366183,23095.146198,22857.297264,...,0.046788,-0.000096,-0.010423,0.000188,0.001216,0.000693,-56.405005,-9.571309,-16.044580,0.664587


In [17]:
tomorrow_input_n = (tomorrow_input - standard_scaler[j][0]) / standard_scaler[j][1]

In [18]:
tomorrow_input_n

,open,high,low,close,Volume,change,Volume_rank,rsi14,close_EMA3,close_EMA10,...,Volume_rank_EMA3_EMA30_offset,Volume_rank_EMA3_EMA200_offset,change_EMA3,change_EMA10,change_EMA30,change_EMA200,change_EMA3_EMA10_offset,change_EMA3_EMA30_offset,change_EMA3_EMA200_offset,close_rel
5054,1.532456,1.516141,1.551297,1.511187,-0.450732,-0.307219,-1.649838,-1.156601,1.537870,1.578700,...,0.092610,-0.744657,-0.577272,-0.624838,-0.824322,-1.095350,-0.018342,0.022589,0.036729,1.511187
5055,1.525415,1.505715,1.508436,1.479496,-0.279865,-0.437700,-1.276668,-1.332739,1.510008,1.562583,...,0.421898,-0.479497,-0.661948,-0.763073,-0.926363,-1.142725,-0.018670,0.022513,0.037381,1.479496
5056,1.479643,1.470962,1.504865,1.475974,-0.398306,-0.095588,-1.721602,-1.352568,1.494314,1.548753,...,-0.020868,-0.795404,-0.412455,-0.679334,-0.900663,-1.144690,-0.019707,0.020426,0.025663,1.475974
5057,1.490206,1.481388,1.515580,1.475974,-0.493589,-0.052248,-2.146474,-1.352568,1.486467,1.537437,...,-0.789889,-1.378550,-0.250739,-0.585885,-0.861274,-1.140943,-0.020478,0.019038,0.018233,1.475974
5058,1.479643,1.453586,1.476291,1.447803,-0.429677,-0.399406,-1.831147,-1.525231,1.468441,1.523035,...,-0.709159,-1.342310,-0.466015,-0.709168,-0.947369,-1.182834,-0.019506,0.020673,0.026539,1.447803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5169,2.173428,2.177232,2.204869,2.166237,-0.261285,-0.087112,-1.373714,0.736028,2.128433,2.047536,...,1.814682,0.471415,0.631770,0.792852,0.366789,-0.335985,-0.019320,0.023621,0.039802,2.166237
5170,2.166296,2.149074,2.150608,2.187635,-0.363161,0.157153,-1.962976,0.834156,2.159770,2.075491,...,0.863568,-0.175704,0.449999,0.739114,0.398566,-0.312779,-0.019812,0.021850,0.031340,2.187635
5171,2.187692,2.149074,2.179547,2.130573,-0.401591,-0.607111,-2.033672,0.403901,2.146875,2.087943,...,0.292712,-0.571362,-0.292825,0.255415,0.157633,-0.390193,-0.023215,0.014881,0.001228,2.130573
5172,2.109242,2.117397,2.096347,2.073511,-0.272914,-0.616657,-1.340857,0.012400,2.111863,2.087711,...,0.973703,-0.023103,-0.672380,-0.145832,-0.071137,-0.468091,-0.115779,0.002447,-0.023327,2.073511


In [22]:
a = core.np.arange(6).reshape(2,3) + 10
a

array([[10, 11, 12],
       [13, 14, 15]])

In [23]:
core.np.argmax(tomorrow_prediction)

0